In [1]:
# importing libraires 
import numpy as np
import pandas as pd 
import os 
import tensorflow as tf 
import  librosa as lb 


from tensorflow.keras.utils import to_categorical 
from tensorflow.keras import Sequential , layers 

from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix , classification_report 
import IPython.display as ipd

In [2]:
data = pd.read_csv('../input/urbansound8k/UrbanSound8K.csv')

In [3]:
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8732 entries, 0 to 8731
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   slice_file_name  8732 non-null   object 
 1   fsID             8732 non-null   int64  
 2   start            8732 non-null   float64
 3   end              8732 non-null   float64
 4   salience         8732 non-null   int64  
 5   fold             8732 non-null   int64  
 6   classID          8732 non-null   int64  
 7   class            8732 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 545.9+ KB


In [5]:
# preprocessing 
classes = data.groupby('classID')['class']
classes.unique()

classID
0     [air_conditioner]
1            [car_horn]
2    [children_playing]
3            [dog_bark]
4            [drilling]
5       [engine_idling]
6            [gun_shot]
7          [jackhammer]
8               [siren]
9        [street_music]
Name: class, dtype: object

In [6]:
# data['class'].unique()

In [7]:
def feature_extract(path):
    data , sample = lb.load(path)
    data = lb.feature.mfcc(data , n_mfcc = 128)
    data = np.mean(data , axis = 1)
    return data

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



x = []
y = [] 

for i in range(len(data)):
    path = "../input/urbansound8k/" + "fold" + str(data.loc[i , 'fold']) +"/" + str(data.loc[i , "slice_file_name"])
    x.append(feature_extract(path))
    y.append(data.loc[i , 'classID'])
    if(i%50 == 0):
        print(i)
        

X = np.array(x)

y = np.array(y)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550


/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1323
  return f(*args, **kwargs)


3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300


/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1103
  return f(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1523
  return f(*args, **kwargs)


8350
8400
8450
8500
8550
8600
8650
8700


In [9]:
y = to_categorical(y)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state = 40)

In [11]:
model = Sequential([
    layers.Dense(800 , activation = 'relu' , input_shape = (128 , )),
    layers.Dense(500 , activation = 'relu'),
    layers.Dense(200 , activation = 'relu'),
    layers.Dense(100 , activation = 'relu'),
    layers.Dense(50 , activation = 'relu'),
    layers.Dense(10 , activation = 'softmax')
])

2022-09-23 10:23:54.565976: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 10:23:54.722827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 10:23:54.727155: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 10:23:54.732752: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 800)               103200    
_________________________________________________________________
dense_1 (Dense)              (None, 500)               400500    
_________________________________________________________________
dense_2 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_3 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
Total params: 629,560
Trainable params: 629,560
Non-trainable params: 0
__________________________________________________

In [13]:
model.compile(loss = 'categorical_crossentropy' ,
              optimizer = 'adam' ,
              metrics = ['accuracy'])

In [14]:
history = model.fit(X_train , y_train , epochs = 25)

2022-09-23 10:23:58.707241: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
219/219 [==============================] - 3s 3ms/step - loss: 1.7380 - accuracy: 0.4561
Epoch 2/25
219/219 [==============================] - 1s 3ms/step - loss: 0.9908 - accuracy: 0.6736
Epoch 3/25
219/219 [==============================] - 1s 3ms/step - loss: 0.7458 - accuracy: 0.7569
Epoch 4/25
219/219 [==============================] - 1s 3ms/step - loss: 0.5602 - accuracy: 0.8159
Epoch 5/25
219/219 [==============================] - 1s 3ms/step - loss: 0.4577 - accuracy: 0.8485
Epoch 6/25
219/219 [==============================] - 1s 3ms/step - loss: 0.3625 - accuracy: 0.8799
Epoch 7/25
219/219 [==============================] - 1s 3ms/step - loss: 0.3305 - accuracy: 0.8901
Epoch 8/25
219/219 [==============================] - 1s 3ms/step - loss: 0.2967 - accuracy: 0.8966
Epoch 9/25
219/219 [==============================] - 1s 5ms/step - loss: 0.2322 - accuracy: 0.9217
Epoch 10/25
219/219 [==============================] - 1s 3ms/step - loss: 0.1883 - accuracy: 0.9354

In [15]:
y_true = np.argmax(y_test , axis = 1)
y_pred = np.argmax(model.predict(X_test) , axis = 1)

print(confusion_matrix(y_true , y_pred))

[[186   0   3   0   0   0   0   0   0   3]
 [  0  70   1   0   0   0   0   1   0   5]
 [  1   2 173   7   1   4   1   1   1  11]
 [  1   0   9 180   2   4   4   0  11   8]
 [  2   0   0   0 182   0   0   0   0   1]
 [  0   0   0   0   0 205   0   0   0   1]
 [  0   0   3   5   0   0  53   0   1   1]
 [  0   0   1   0   3   0   0 190   0   2]
 [  0   1   1   1   1   3   0   0 182   2]
 [  3   2   7   1   3   1   0   1   1 197]]


In [16]:
y_test.shape

(1747, 10)

In [17]:
class_names = data.groupby('classID')['class']
class_names.unique()

classID
0     [air_conditioner]
1            [car_horn]
2    [children_playing]
3            [dog_bark]
4            [drilling]
5       [engine_idling]
6            [gun_shot]
7          [jackhammer]
8               [siren]
9        [street_music]
Name: class, dtype: object

In [18]:
class_names = pd.DataFrame(class_names.unique() )
class_names

,class
classID,
0,[air_conditioner]
1,[car_horn]
2,[children_playing]
3,[dog_bark]
4,[drilling]
5,[engine_idling]
6,[gun_shot]
7,[jackhammer]
8,[siren]


In [19]:
test_path = '../input/urbansound8k/fold9/101729-0-0-38.wav'
feature = np.array([feature_extract(test_path)])
ID = np.argmax(model.predict(feature)[0])
print(ID)
print(class_names.loc[ID , 'class'])
ipd.Audio(test_path)

0
['air_conditioner']
